<a href="https://colab.research.google.com/github/Wpert/numerics/blob/main/defineIntegrals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Определённые интегралы

Итак, пора бы начаучиться считать определенные интегралы. Пока что не буду ставить своей целью математически сторого обосновывать все асимптотики и точности, с которыми приближает значнеия тот или иной метод. В рамках изучения методов я так или иначе сам всё примерно запомню. А в этом файлике чтобы сделать меньше ошибок думаю стоит оставить всё как есть.

Сперва найдём среди использованных ранее мной библиотек ту, которая позволит мне считать интегралы (sympy). И научимся их вычислять, мне надо достаточно долго поиграться чтобы осознать что происходит.

После от меня требуется написать несколько алгоритмов, которые будут высчитывать интегралы. И придумать какие-нибудь тесты, чтобы была видна точность каждого из алгоритмов.

Всё это проводится так как есть большое желание написать разложение какой-нибудь функции в триг. ряд Фурье на плюсах. Но пока ещё не имею достаточного опыта в численных расчетах.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from typing import Tuple, List, Union
sp.init_printing()

from sympy import log, exp, oo, cos, sin, pi
from sympy.abc import x

expr_1 = exp(-x**2)
expr_2 = 1 / x ** 2
expr_3 = cos(x) + 2 * sin(x)
expr_4 = cos(2 * x) * cos(x)
expr_5 = 2 * sin(x) * cos(x) * cos(2 * x)
expr_6 = sin(x)

display( sp.integrate(expr_1, (x, -oo, oo)) )
display( sp.integrate(expr_2, (x, 1, oo)) )
display( sp.integrate(expr_3, (x, 0, 2 * pi )) )
display( sp.integrate(expr_4, (x, 0, pi)) )
display( sp.integrate(expr_5, (x, 0, pi / 4)) )
display( sp.integrate(expr_6, (x, 0, pi)) )

def ResidualInt(expr, func, interval: Tuple[float], eps) -> None:
  from sympy.abc import x
  a = interval[0]
  b = interval[1]
  solution = sp.integrate(expr, (x, a, b))

  # return np.abs(solution - 0.0)

√π

1

0

0

1/4

2

Сперва попробую реализовать самый тривиум подсчета интеграла. Без подсчета мне кажется, что подсчет через подход Дарбу будет малоэффективен даже с вида самой формулы. (Искать минимумы, максимумы на отрезках вообще будто бы не для вычислений придумывалось)

На прямой подсчет с помощью формулы Ньютона-Лейбница я ещё не дорос, так как даже и близко не представляю как запрограммировать трюки интегрирования. Вероятно каким-то образом надо выразить все элементраные функции через одну хорошую и считать интегралы отталкиваясь от неё. Может быть разложить в какой-то функциональный ряд и интегрировать на радиусе сходимости.

Потому пока сделаем по определению интеграла Римана разбиение, на каждом отрезке отмечаем каким-нибудь образом точку и тогда запишем интегральную сумму следующим образом.

$$\sigma (f, P, \xi) = \sum_i f(\xi_i) \cdot |I_i|$$

Значит сейчас буду пробовать отмечать точку рандомом или брать середину отрезка.

Сходу можно сказать, что вероятностный выбор точки будет работать медленнее, чем деление, а погрешность непредсказуемой как раз из-за рандома, когда в то же время подсчет прямоугольников с высотой равной $f(\frac{a + b}{2})$ намного проще и более предсказуем с заданной функцией.

Также можно реализовать подсчёт методом трапеций:
$$\sigma (f, P, \xi) = \sum_i \frac{f(a_i) + f(b_i)}{2} \cdot I_i$$

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from typing import Tuple, List, Callable
import typing
import math as mt

def RandomRiemannIntegral(func: Callable, from_: float, to_: float, N: int) -> float:
  a_b = np.linspace(from_, to_, N)
  dx = (to_ - from_) / (N - 1)
  xi = [np.random.uniform(a_b[i], a_b[i + 1]) for i in range(N - 1)]

  sum = 0
  for xi_ in xi:
    sum += func(xi_)

  sum *= dx
  return sum

# the same code tbh
def MiddleRiemannIntegral(func: Callable, from_: float, to_: float, N: int) -> float:
  a_b = np.linspace(from_, to_, N)
  dx = (to_ - from_) / (N - 1)
  xi = [(a_b[i] + a_b[i + 1]) / 2 for i in range(N - 1)]

  sum = 0
  for xi_ in xi:
    sum += func(xi_)

  sum *= dx
  return sum

def TrapezoidRiemannIntegral(func: Callable, from_: float, to_: float, N: int) -> float:
  a_b = np.linspace(from_, to_, N)
  dx = (to_ - from_) / (N - 1)
  sum = 0

  for i in range(N - 1):
    sum += func(a_b[i]) + func(a_b[i + 1])

  sum /= 2
  sum *= dx
  return sum

def TestAppr(*args: List[Callable]):
  from sympy import integrate, oo, pi
  from sympy.abc import x

  def Test(func: Callable, N: int, *args: List[Callable]):
    testFunc = func
    testExpr = func(x)
    a, b = (0, float(pi.evalf()))

    rightValue = sp.integrate(testExpr, (x, a, b))
    print(f'Right value {rightValue}')

    for intFunc in args:
      value = intFunc(testFunc, a, b, N)
      print(intFunc.__name__)
      print(np.abs(rightValue - value))
    print()

    # integral_0^π sqrt(x)dx = (2 π^(3/2))/3 ≈ 3.71221866455447
    # integral_0^π e^x dx = e^π - 1 ≈ 22.1406926327793
  functions = [sp.sqrt, sp.exp, sp.cos]
  for pwr in range(1, 3):

    N = 20 * 10 ** pwr
    print(f'N = {N}')
    for func in functions:
      print(f"---{func.__name__}Test---")
      Test(func, N, *args)

TestAppr(RandomRiemannIntegral, MiddleRiemannIntegral, TrapezoidRiemannIntegral)

N = 200
---sqrtTest---
Right value 3.71221866455447
RandomRiemannIntegral
0.000768493438111584
MiddleRiemannIntegral
0.000117846493902007
TrapezoidRiemannIntegral
0.000406495879285096

---expTest---
Right value 22.1406926327793
RandomRiemannIntegral
0.0148309106190396
MiddleRiemannIntegral
0.000229916636175886
TrapezoidRiemannIntegral
0.000459834704884088

---cosTest---
Right value 1.22464679914735E-16
RandomRiemannIntegral
0.00151371394065302
MiddleRiemannIntegral
1.64529425619362e-16
TrapezoidRiemannIntegral
9.61742138493436e-17

N = 2000
---sqrtTest---
Right value 3.71221866455447
RandomRiemannIntegral
1.74549826175152e-5
MiddleRiemannIntegral
3.76447415151304e-6
TrapezoidRiemannIntegral
1.28937743664181e-5

---expTest---
Right value 22.1406926327793
RandomRiemannIntegral
0.000435093737756631
MiddleRiemannIntegral
2.27852650880322e-6
TrapezoidRiemannIntegral
4.55705318103128e-6

---cosTest---
Right value 1.22464679914735E-16
RandomRiemannIntegral
2.04204552662454e-5
MiddleRiemannInt

Здесь я посчитаю площадь, приближая функцию параболами. Тогда нам дана функция $f(x)$, отрезок $I_i = [a, b]$.

Значит чтобы построить параболу $\alpha x^2 + \beta x + \gamma = 0$ требуется найти её коэффициенты, при чем мы можем узнать значения функции от аргумента.

Построим следующую систему уравнений
\begin{array}{rcl}
  \alpha a^2 + \beta a + \gamma & = f(a) \\
  \alpha b^2 + \beta b + \gamma & = f(b) \\
  \alpha c^2 + \beta c + \gamma & = f(c)
\end{array}
Которая эквивалентна следующей записи

$$ \begin{pmatrix} a^2 & a & 1 \\ b^2 & b & 1 \\ c^2 & c & 1 \end{pmatrix} \times \begin{pmatrix} \alpha \\ \beta \\ \gamma \end{pmatrix} = \begin{pmatrix} f(a) \\ f(b) \\ f(c) \end{pmatrix}$$

<!-- $\left[\begin{array}{c} x_1 \\ x_2 \end{array} \right] = \begin{bmatrix} A & B \\ C & D \end{bmatrix} \times \left[ \begin{array}{c} y_1 \\ y_2 \end{array} \right]$ -->

А как решать подобные системы мы уже знаем, но я предпочту использовать np.linalg.solve(), чтобы не нагружать очень блоки кода. Естественно, можно сразу сказать, что данное решение мало того что очень затратно по времени, так ещё и по памяти.

Теперь, зная коэффициенты, можно тривиально посчитать площадь под графиком.
$$ S = \int_a^b \alpha x^2 + \beta x + \gamma \,dx = (\frac{\alpha}{3} x^3 + \frac{\beta}{2} x^2 + \gamma x)\Bigg|_a^b $$

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from sympy import integrate, log, exp, oo, cos, sin, pi
from typing import Tuple, List, Callable
import typing
import math as mt

def calcPolySurface(coeffs: np.array, from_, to_) -> float:
  a_ = float(coeffs[0]) / 3
  b_ = float(coeffs[1]) / 2
  c_ = float(coeffs[2])

  sum_b = ((a_ * to_ + b_) * to_ + c_) * to_
  sum_a = ((a_ * from_ + b_) * from_ + c_) * from_

  return sum_b - sum_a

def makeVandermondMtrx(b: np.array) -> np.array:
  N = b.shape[0]
  A = np.ones((N, N))
  for i in range(N - 1):
    A[:, N - 1 - i - 1] = (A[:, N - i - 1].reshape(3,1) * b).reshape(3,)
  return A

def ParabolicRiemannIntegral(func: Callable, from_: float, to_: float, N: int) -> float:
  a_b = np.linspace(from_, to_, N)
  sum = 0

  for i in range(N - 1):
    xi = np.array([[a_b[i]],
                   [(a_b[i] + a_b[i + 1]) / 2],
                   [a_b[i + 1]]])

    A = makeVandermondMtrx(xi)
    F = np.vectorize(func)(xi).reshape(3, 1)
    F = np.vectorize(float)(F).reshape(3, 1)

    coeffs = np.linalg.solve(A, F)
    sum += calcPolySurface(coeffs.reshape(3), a_b[i], a_b[i + 1])

  return sum

# print(calcSquarePoly((0, 0, 1), 0, 2))

print(ParabolicRiemannIntegral(sp.sqrt, 0, float(pi.evalf()), 200))
    # integral_0^π sqrt(x)dx = (2 π^(3/2))/3 ≈ 3.71221866455447
b = np.array([[4],
              [9],
              [15]])
A = makeVandermondMtrx(b)
print(A)
F = np.array([[15],
              [80],
              [224]])
print(np.linalg.solve(A, F))

3.7121617302573133
[[ 16.   4.   1.]
 [ 81.   9.   1.]
 [225.  15.   1.]]
[[ 1.00000000e+00]
 [-5.55111512e-16]
 [-1.00000000e+00]]


А тут можно попытаться реализовать метод Симпсона (метод парабол). Для начала надо вывести его аналитически, идея всё та же, что и в алгоритме сверху, нам дан отрезок $I_i = [a, b]$, мы берём три точки $a, b, m = \frac{a + b}{2}$ и подбираем такой многочлен, который будет проходить сквозь эти точки.

Такое чувство, будто метод выше и этот не будут отличаться по точности совершенно никак, все-таки я делаю абсолютно одно и то же, просто разными способами. Метод Симпсона будет разве что быстрее в несколько раз.

Функцию можно приблизить интерполяционным многочленом Лагранжа, представив её в виде:
$$\begin{align*} f(x) &\approx P_2(x) = f(a) \frac{(x - m)(x - b)}{(a - m)(a - b)} + f(m) \frac{(x - a)(x - b)}{(m - a)(m - b)} + f(b) \frac{(x - a)(x - m)}{(b - a)(b - m)} =\\
&= 2 f(a) \frac{x^2 - (m + b)x + mb}{(a - b)(a - b)} + 4 f(m) \frac{x^2 - (a + b)x + ab}{(b - a) (a - b)} + 2 f(b) \frac{x^2 - (a + m)x + am}{(b - a)(b - a)} \\
&= \frac{2}{(a - b)^2} \Bigl(f(a)(x^2 - (m + b)x + mb) - 2f(m)(x^2 - (a + b)x + ab) + f(b) (x^2 - (a + m)x + am)\Bigr) \end{align*}$$

Тогда, можно запросто взять интеграл $\int_a^b P_2(x)dx$ и получить требуемый ответ. Честно посчитаю этот интеграл.

$$\begin{align*}
\int_a^b P_2(x) =& \int_a^b \frac{2}{(a - b)^2} \Bigl(f(a)(x^2 - (m + b)x + mb) - 2f(m)(x^2 - (a + b)x + ab) + f(b) (x^2 - (a + m)x + am)\Bigr)
\\
=& \frac{2}{(a - b)^2} \Bigl(\frac{f(a)}{3}x^3 - \frac{f(a)(m + b)}{2} x^2 + f(a)mbx - \frac{2f(m)}{3}x^3 + f(m)(a + b)x^2 + 2af(m)bx + \frac{f(b)}{3}x^3 - \frac{f(b)(a + m)}{2} x^2 + amf(b)x \Bigr)\Bigg|_a^b
\\
=& \frac{2}{(a - b)^2}\Bigl( \frac{f(a) - 2f(m) + f(b)}{3}x^3 + \frac{2f(m)(a + b) - f(a)(m + b) - f(b) (a + m)}{2} x^2 + (f(a)mb + 2af(m)b + amf(b))x \Bigr) \Bigg|_a^b
\\
=& \frac{2}{a - b} \Bigl( \frac{f(a) - 2f(m) + f(b)}{3} (b^2 - ab + a^2) + \frac{2f(m)(a + b) - f(a)(m + b) - f(b) (a + m)}{2} (a + b) + (f(a)mb + 2af(m)b + amf(b)) \Bigr)
\\
=& \frac{2}{a - b} \Bigl( f(a)\bigl( \frac{b^2 + ab + a^2}{3} - \frac{(m + b) (a + b)}{2} + mb \bigr) - 2f(m) \bigl( \frac{b^2 - ab + a^2}{3} - \frac{a^2 + 2ab + b^2}{2} - ab \bigr) + f(b) \bigl( \frac{b^2 + ab + a^2}{3} - \frac{(a + m)(a + b)}{2} + am \bigr) \Bigr)
\\
=& \frac{1}{3(a - b)} \Bigl( f(a) \bigl( 2b^2 + 2ab +2a^2 - 3(\frac{a^2 + b^2}{2} + 2ab + b^2) + 3ab + 3b^2 \bigr) - 2f(m) \bigl( 2b^2 - 2ab + 2a^2 - 3a^2 - 3b^2 \bigr) + f(b) \bigl( 2b^2 + 2ab +2a^2 - 3(\frac{a^2 + b^2}{2} + a^2 + 2ab) + 3a^2 + 3ab \bigr) \Bigr)
\\
=& \frac{1}{3(a - b)} \Bigl( f(a)(\frac{1}{2}b^2 - ab + \frac{1}{2}a^2) + 2f(m)(a^2 - 2ab + b^2) + f(b)(\frac{1}{2}b^2 - ab + \frac{1}{2}a^2) \Bigr)
\\
=& \frac{1}{6(a - b)} \Bigl( f(a)(a - b)^2 + 4f(m)(a - b)^2 + f(b)(a - b)^2 \Bigr)
\\
=& \frac{a - b}{6} \Bigl( f(a) + 4f(m) + f(b) \Bigr)
\end{align*}$$

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from sympy import oo, pi
from typing import Tuple, List, Callable
import typing
import math as mt

def SimpsonsRiemannIntegral(func: Callable, from_: float, to_: float, N: int) -> float:
  a_b = np.linspace(from_, to_, N)
  dx = (to_ - from_) / (N - 1)
  sum = 0

  for i in range(N - 1):
    m = (a_b[i] + a_b[i + 1]) / 2
    sum += func(a_b[i]) + 4 * func(m) + func(a_b[i + 1])

  sum *= dx
  sum /= 6
  return sum

Метод Гаусса-Лежандра было бы прекрастно реализовать, уже "классическая" для меня задача: дан отрезок $I_i = [a, b]$, найти на нем значение $\int_a^b f(x) dx$.

Метод является наиболее точным из всех, что я здесь реализовывал, он также не требует невероятных вычислений, как в геом решении для интерполяционных полиномов выше.

И благодаря некоторой "магии" выясняется, что:

$$\int_a^b f(x) dx = \frac{b - a}{2} \sum_i w_i f(\frac{a + b}{2} + \frac{a - b}{2}x_i) \\
w_i = \frac{1}{(1 - x_i^2) (P_n'(x_i))^2}$$

Где $x_i - i$-тый корень полинома Лежандра $P_n$.

Но сперва требуется понять почему все это работает изначально. Увы, но это приключение затянулось надолго, потому желания делать огромные выкладки прямо здесь у меня нет. Сделаю доп "лабу" по полиномам Лежандра.



In [19]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from sympy import oo, pi
from typing import Tuple, List, Callable
import typing
import math as mt

def GaussLegendreIntegral(func: Callable, from_: float, to_: float, N: int, highPrecision=False) -> float:
  # 2nd and 6th degree
  a_b = np.linspace(from_, to_, N)
  sum = 0

  roots = (- 1 / mt.sqrt(3), 1 / mt.sqrt(3))
  weights = (1, 1)

  if (highPrecision):
    # calculate roots of Legendre polynomials
    roots = ()

    # then weights
    weights = ()
    # w_i = 1 / ((1 - x**2) * () ** 2)

  for i in range(N - 1):
    a = a_b[i]
    b = a_b[i + 1]

    for (xi, wi) in zip(roots, weights):
      sum += wi * func((a + b) / 2 + xi * (b - a) / 2 )

  sum *= (to_ - from_) / (N - 1)
  sum /= 2

  return sum

# print(GaussLegendreIntegral(mt.sqrt, 0, float(pi.evalf()), 200))
TestAppr(ParabolicRiemannIntegral, SimpsonsRiemannIntegral, GaussLegendreIntegral)

N = 200
---sqrtTest---
Right value 3.71221866455447
ParabolicRiemannIntegral
5.69342971585840e-5
SimpsonsRiemannIntegral
5.69342971608044e-5
GaussLegendreIntegral
1.44631131502315e-5

---expTest---
Right value 22.1406926327793
ParabolicRiemannIntegral
4.77491823858145e-10
SimpsonsRiemannIntegral
4.77513140140218e-10
GaussLegendreIntegral
3.18344461902598e-10

---cosTest---
Right value 1.22464679914735E-16
ParabolicRiemannIntegral
2.89802224147763e-15
SimpsonsRiemannIntegral
1.78999330968423e-16
GaussLegendreIntegral
1.46574422821106e-16

N = 2000
---sqrtTest---
Right value 3.71221866455447
ParabolicRiemannIntegral
1.78827534602632e-6
SimpsonsRiemannIntegral
1.78827535579629e-6
GaussLegendreIntegral
4.54278529016250e-7

---expTest---
Right value 22.1406926327793
ParabolicRiemannIntegral
8.17124146124115e-14
SimpsonsRiemannIntegral
4.97379915032070e-14
GaussLegendreIntegral
1.42108547152020e-14

---cosTest---
Right value 1.22464679914735E-16
ParabolicRiemannIntegral
6.84011806943619e-14


Здесь я не расписывал